## 예제를 통해 알아보는 Pytorch Geometric 5 basic concepts

### 1. 그래프의 데이터 핸들링

- 노드집합 V, 간선집합 E => 그래프 G = (V,E)
- 하나의 그래프는 torch_geometric.data.Data 라는 클래스로 표현됨
- data.x : 노드 특징 행렬
    - [num_nodes, num_node_features]
- data.edge_index : 그래프의 연결성 (이웃한 노드 수)
    - [2, num_edges]
- data.edge_attr : 엣지 특징 행렬
    - [num_edges, num_edge_features]
- data.y : 학습하고 싶은 대상 (타겟)
    - 그래프 레벨 → [num_nodes, *]
    - 노드 레벨 → [1, *]
- data.pos : 노드 위치 행렬
    - [num_nodes, num_dimensions]


In [18]:
# torch_geometric은 기존 torchvision의 (이미지, 타겟) 튜플 형태보다 조금 더 직관적인 형태
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0,1,1,2],[1,0,2,1]], dtype = torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float) # x는 노드가 갖는 값임 (특징값 행렬)
data = Data(x=x, edge_index=edge_index) # 이런식으로 그래프 정의!
data

Data(x=[3, 1], edge_index=[2, 4])

- edge_index : (2,4) 크기의 행렬 → 2개의 엣지들
    - 노드의 연결성을 의미하는거라 **각 노드 입장에서 이웃한 노드의 수 총합** & 이웃개념이라 처음은 항상 2, 인듯
- x : (3,1) 크기의 행렬 → 3개의 노드와 각 노드의 특징은 단일 값
- ![graph](./graph.png)

In [19]:
edge_index = torch.tensor([[0, 1],
                        [1, 0],
                        [1, 2],
                        [2, 1]], dtype=torch.long)
# 위와 같이 좀 더 직관적으로 입력해 줄 수 있음! (보통 엣지는 노드의 순서쌍으로 나타내는 경우가 많으니까)

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous()) # 이런 경우 contiguous()를 사용해서 동일한 그래프로 표현가능
data

Data(x=[3, 1], edge_index=[2, 4])

추가적으로 다음과 같은 함수들을 제공함
- data.keys : 해당 속성 이름
- data.num_nodes : 노드 총 개수
- data.num_edges : 엣지 총 개수
- data.contains_isolated_nodes() : 고립 노드 여부 확인
- data.contains_self_loops() : 셀프 루프 포함 여부 확인
- data.is_directed() : 그래프의 방향성 여부 확인

### 2. 공통 벤치마크 데이터셋


In [20]:
# ENZYMES 데이터셋을 불러오는 예제
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/data/ENZYMES', name='ENZYMES')
dataset

ENZYMES(600)

In [21]:
print(f'그래프의 개수 : {len(dataset)}')
print(f'그래프의 클래스 종류 : {dataset.num_classes}')
print(f'노드의 특징 수 : {dataset.num_node_features}')

그래프의 개수 : 600
그래프의 클래스 종류 : 6
노드의 특징 수 : 3


In [22]:
# 6종류의 클래스, 600개 그래프 => 수 많은 그래프 중 일부만 갖고오기 : 인덱싱
data = dataset[0]
data # 600개 그래프 중 첫번째 그래프에 대한 정보를 출력

Data(edge_index=[2, 168], x=[37, 3], y=[1])

- edge_index=[2, 168] → 총 84개의 엣지
- x=[37, 3] → 총 37개의 노드와 3개의 노드 특성
- y=[1] → 그래프 레벨 타겟

In [23]:
print(data.is_undirected)
train_dataset = dataset[:540]
test_dataset = dataset[540:]
dataset.shuffle() # 데이터셋 셔플
dataset

<bound method BaseData.is_undirected of Data(edge_index=[2, 168], x=[37, 3], y=[1])>


ENZYMES(600)

In [24]:
# Cora 데이터셋 불러오기
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/data/Cora', name='Cora')
# 데이터셋 전체가 하나의 그래프임
print(f'그래프의 개수 : {len(dataset)}')
print(f'그래프의 클래스 종류 : {dataset.num_classes}')
print(f'노드의 특징 수 : {dataset.num_node_features}')

그래프의 개수 : 1
그래프의 클래스 종류 : 7
노드의 특징 수 : 1433


### 3. 미니배치

- 신경망은 주로 배치단위로 학습을 하는데
- 기존은 torch.utils.data.DataLoader 로 수행하였다면
- geometric에서는 torch_geometric.data.DataLoader로 수행한다 (뭐 똑같음 별거없다)

In [28]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/data/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for idx, batch in enumerate(loader):
    print(f'batch 정보 : {batch}')
    print(f'하나의 batch에서 처리되는 그래프 수 : {batch.num_graphs}\n')
    if idx == 2 : break

batch 정보 : DataBatch(edge_index=[2, 4040], x=[1104, 21], y=[32], batch=[1104], ptr=[33])
하나의 batch에서 처리되는 그래프 수 : 32

batch 정보 : DataBatch(edge_index=[2, 3764], x=[973, 21], y=[32], batch=[973], ptr=[33])
하나의 batch에서 처리되는 그래프 수 : 32

batch 정보 : DataBatch(edge_index=[2, 3738], x=[949, 21], y=[32], batch=[949], ptr=[33])
하나의 batch에서 처리되는 그래프 수 : 32



c:\Users\SeoJeongBin\miniconda3\envs\kotorch\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


### 4. 데이터 변환
- 마찬가지로 기존과 거의 유사하게 torch_geometric.transforms 로 전처리 수행
- torchvision과 동일하게 torch_geometric.transfomrs.Compose로 다양한 변환함수 구성
    - 다만 위에서 살펴본 Data객체에 특화된 변환함수임

In [29]:
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/data/ShapeNet', categories=['Airplane'])

print(f'그래프 총 개수 : {len(dataset)}')
print(f'첫번째 그래프 정보 : {dataset[0]}')

Extracting \data\ShapeNet\shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...


그래프 총 개수 : 2349
첫번째 그래프 정보 : Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])


Done!


=> edge_index가 없다는건 링크가 없고 노드만 있는 점 구름 형태의 데이터셋 의미

In [30]:
# 그래프 데이터 변환하기
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/data/ShapeNet', categories=['Airplane'],
                   pre_transform=T.KNNGraph(k=6),
                   transform=T.RandomTranslate(0.01))

dataset[0]

c:\Users\SeoJeongBin\miniconda3\envs\kotorch\lib\site-packages\torch_geometric\data\dataset.py:150: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, make sure to sure to delete '\data\ShapeNet\processed' first
  warnings.warn(


Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])

- pre_transform = T.KNNGraph(k=6) : KNN을 통해 데이터를 그래프 형태로 변형합니다.
    - 결괏값으로 edge_index가 추가된 것을 확인할 수 있습니다. (즉, 연결상태 생성)
    - pre_transform은 디스크 메모리에 할당하기 전에 적용합니다.
- transform = T.RandomTranslate(0.01) : 각 노드의 위치를 아주 조금 이동시킵니다.
    - 일종의 perturbation 작업이라고 보시면 됩니다.


### 5. 그래프로 학습하기
- 간단한 GCN layer 구성한 뒤, Cora 데이터셋에 적용하는 예제
- Graph node classification 작업임 (node 속성 분류 작업 - 예>user의 특성 분류)
- 데이터셋 설명
    - Citation Network인데 각 논문을 노드로 보고 인용 관계가 엣지임
    - 논문마다 등장하는 1433개의 특정단어를 모아 단어사전으로 만들고 논문마다 단어 사잔의 등장여부를 feature vector로 만들어서 노드의 특징을 만듦
    - 해당 데이터셋에 적용할 grpah node classificaion은 임의의 논문에 대해 논문에 등장한 단어들과 인용 관계만ㅇ르 통해 어떤 종류의 논문인지 맞히는 작업임!


In [33]:
from torch_geometric.datasets import planetoid

dataset = Planetoid(root='data/Cora', name='Cora')
dataset

Processing...
Done!


Cora()

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init(self):
        super(Net, self).__init__()
        self.conv1 = 